In [86]:
class Conv2d:
    def __init__(
        self, in_channels, out_channels, kernel_size, stride=1, padding=0, bias=True
    ):
        self.kernel_size = kernel_size
        self.kernel = torch.randn(
            out_channels, in_channels, kernel_size, kernel_size
        )
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.stride = stride
        self.bias = torch.randn(out_channels) if bias else None
        self.padding = padding

    def forward(self, x):
        n, c, h, w = x.shape
        # Padding for top and bottom
        verticalPadding = torch.zeros(
            (n, c, self.padding, w), device=x.device
        )
        x = torch.cat(
            (verticalPadding, x, verticalPadding), dim=2
        )  # Concatenate along height dimension
        # Padding for left and right
        horizontalPadding = torch.zeros(
            (n, c, (h + self.padding * 2), self.padding), device=x.device
        )
        x = torch.cat(
            (horizontalPadding, x, horizontalPadding), dim=3
        )  # Concatenate along width dimension
        out_width = (w + 2 * self.padding - self.kernel_size) // self.stride + 1
        out_heigh = (h + 2 * self.padding - self.kernel_size) // self.stride + 1
        out = torch.zeros(
            (n, self.out_channels, out_heigh, out_width), device=x.device
        )
        for ni in range(n):
            for hi in range(out_heigh):
                hi_i = hi * self.stride
                for wi in range(out_width):
                    wi_i = wi * self.stride
                    for ci in range(self.out_channels):
                        out[ni, ci, hi, wi] = (
                            x[ni, :, hi_i : hi_i + self.kernel_size, wi_i : wi_i + self.kernel_size]
                            * self.kernel[ci]
                        ).sum()
        return out

    def __call__(self, x):
        return self.forward(x)

In [87]:
a = torch.ones((1, 3, 3, 3))
con = Conv2d(3, 1, 3, 1, 1)
b = con(a)
print(b.shape)

tensor([[[[0., 0., 0., 0., 0.],
          [0., 1., 1., 1., 0.],
          [0., 1., 1., 1., 0.],
          [0., 1., 1., 1., 0.],
          [0., 0., 0., 0., 0.]],

         [[0., 0., 0., 0., 0.],
          [0., 1., 1., 1., 0.],
          [0., 1., 1., 1., 0.],
          [0., 1., 1., 1., 0.],
          [0., 0., 0., 0., 0.]],

         [[0., 0., 0., 0., 0.],
          [0., 1., 1., 1., 0.],
          [0., 1., 1., 1., 0.],
          [0., 1., 1., 1., 0.],
          [0., 0., 0., 0., 0.]]]])
torch.Size([1, 1, 3, 3])


In [3]:
loss = ml.CrossEntropyLoss()
# x = [0.3,0.3,0.4]
# y = 1
x = [0.3,0.3,0.4],[0.1,0.3,0.6],[0.4,0.3,0.3]
y = [1,2,0]
loss(x,y)

0.877029719988694

In [3]:
import torch
class BatchNorm1d:

    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.momentum = momentum
        self.training = True
        # parameters (trained with backprop)
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)
        # buffers (trained with a running 'momentum update')
        self.running_mean = torch.zeros(dim)
        self.running_var = torch.ones(dim)

    def __call__(self, x):
        # calculate the forward pass
        if self.training:
            xmean = x.mean(0, keepdim=True)  # batch mean
            xvar = x.var(0, keepdim=True)  # batch variance
        else:
            xmean = self.running_mean
            xvar = self.running_var
        xhat = (x - xmean) / (xvar + self.eps) ** 0.5
        self.out = self.gamma * xhat + self.beta
        # update the buffers
        if self.training:
            with torch.no_grad():
                self.running_mean = (
                    self.momentum * xmean + (1 - self.momentum) * self.running_mean
                )
                self.running_var = (
                    self.momentum * xvar + (1 - self.momentum) * self.running_var
                )
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]

In [14]:
import torch
class LayerNorm:

    def __init__(self, dim, eps=1e-5, momentum=0.1):
        self.eps = eps
        self.gamma = torch.ones(dim)
        self.beta = torch.zeros(dim)

    def __call__(self, x):
        last_dim = x.ndim - 1
        # calculate the forward pass
        xmean = x.mean(last_dim, keepdim=True)  # batch mean
        xvar = x.var(last_dim, keepdim=True)  # batch variance
        xhat = (x - xmean) / (xvar + self.eps) ** 0.5
        self.out = self.gamma * xhat + self.beta
        return self.out

    def parameters(self):
        return [self.gamma, self.beta]

In [15]:
module = LayerNorm(200)
x = torch.randn(128, 200)
print(x.shape)
x = module(x)

torch.Size([128, 200])


In [16]:
x[:, 0].mean(), x[:, 0].std()

(tensor(0.0326), tensor(1.0027))

In [17]:
x[0, :].mean(), x[0, :].std()

(tensor(-4.7684e-09), tensor(1.0000))

In [89]:
import torch
from torch import nn
input = torch.arange(1, 3, dtype=torch.float32).view(1, 1, 2)
print(input.shape)

m = nn.Upsample(scale_factor=2, mode='nearest')
b = m(input)

print(b.shape)
print(input)
print(b)

torch.Size([1, 1, 2])
torch.Size([1, 1, 4])
tensor([[[1., 2.]]])
tensor([[[1., 1., 2., 2.]]])


In [24]:
from sklearn.metrics import cohen_kappa_score

predictions = [0, 2, 2, 1, 3]
actuals = [0, 2, 2, 1, 1]
print(cpmp_qwk(predictions, actuals, max_rat=3))
print(cohen_kappa_score(predictions, actuals, weights='quadratic'))

0.5454545454545454
0.5454545454545454


In [ ]:
a = ml.Tensor([[[3,4,1],[6,7,5],[9,4,2]]])

In [ ]:
print(a.shape)

[1, 3, 3]


In [ ]:
c = ml.Conv2d(1,3,2)

[Neuron[weights = [tensor(0.7637685560534376)]], Neuron[weights = [tensor(0.06843865354674716)]], Neuron[weights = [tensor(0.41677226609053286)]]]
[Neuron[weights = [tensor(0.2722612669998846)]], Neuron[weights = [tensor(0.9472328319584717)]], Neuron[weights = [tensor(0.2497754153092283)]]]
[Neuron[weights = [tensor(0.962401763507914)]], Neuron[weights = [tensor(0.1980097529178968)]], Neuron[weights = [tensor(0.7574230809474386)]]]
[Neuron[weights = [tensor(0.8886944081513554)]], Neuron[weights = [tensor(0.7740658709531709)]], Neuron[weights = [tensor(0.13666773221265538)]]]


In [ ]:
len(c.kernel)

2

In [ ]:
c(a)

[[tensor(3), tensor(4)], [tensor(6), tensor(7)]]
[[tensor(4), tensor(1)], [tensor(7), tensor(5)]]
[[tensor(6), tensor(7)], [tensor(9), tensor(4)]]
[[tensor(7), tensor(5)], [tensor(4), tensor(2)]]


In [ ]:
b = ml.Tensor([[1,2,3], [2,4,6]])

In [ ]:
import torch
from torch import nn
# With Learnable Parameters
m = nn.BatchNorm2d(100)
input = torch.randn(20, 100, 35, 45)
output = m(input)

In [ ]:
input

tensor([[[[-6.9298e-01,  7.1015e-01, -1.5096e-01,  ...,  1.1431e+00,
           -8.8724e-01, -8.7056e-02],
          [ 9.1351e-01, -6.5648e-01,  1.0586e+00,  ...,  1.0006e+00,
            1.0965e-01, -7.5269e-01],
          [ 1.3521e+00,  9.5729e-01,  1.5397e-01,  ...,  3.4955e-01,
           -3.0766e-02,  1.9866e-01],
          ...,
          [-8.4927e-01,  6.3140e-01, -2.5442e-01,  ..., -8.5770e-01,
            1.3074e-01, -1.9427e+00],
          [ 1.9148e+00,  3.1475e+00,  1.7911e-01,  ..., -1.9436e+00,
            3.4487e-01,  1.2056e+00],
          [-9.1049e-03,  5.3780e-01,  1.5040e+00,  ..., -1.7275e-01,
           -1.0628e+00, -2.5817e-01]],

         [[ 2.7421e-01, -4.8959e-01,  9.0121e-01,  ...,  9.6102e-01,
           -1.1918e+00, -5.1106e-01],
          [ 8.9671e-01,  5.4164e-01,  3.6196e-01,  ...,  3.9599e-01,
           -8.9039e-01, -1.5504e-01],
          [-6.2156e-01,  8.4530e-01, -1.3577e+00,  ...,  1.2263e+00,
           -5.1535e-01,  1.0602e+00],
          ...,
     

In [ ]:
output

tensor([[[[-6.9963e-01,  7.0836e-01, -1.5573e-01,  ...,  1.1428e+00,
           -8.9457e-01, -9.1611e-02],
          [ 9.1242e-01, -6.6301e-01,  1.0580e+00,  ...,  9.9983e-01,
            1.0578e-01, -7.5955e-01],
          [ 1.3526e+00,  9.5636e-01,  1.5026e-01,  ...,  3.4651e-01,
           -3.5126e-02,  1.9510e-01],
          ...,
          [-8.5647e-01,  6.2933e-01, -2.5955e-01,  ..., -8.6492e-01,
            1.2694e-01, -1.9537e+00],
          [ 1.9172e+00,  3.1541e+00,  1.7548e-01,  ..., -1.9546e+00,
            3.4181e-01,  1.2055e+00],
          [-1.3390e-02,  5.3541e-01,  1.5049e+00,  ..., -1.7760e-01,
           -1.0707e+00, -2.6332e-01]],

         [[ 2.6285e-01, -5.0339e-01,  8.9187e-01,  ...,  9.5187e-01,
           -1.2079e+00, -5.2492e-01],
          [ 8.8735e-01,  5.3115e-01,  3.5089e-01,  ...,  3.8503e-01,
           -9.0547e-01, -1.6777e-01],
          [-6.3578e-01,  8.3578e-01, -1.3742e+00,  ...,  1.2180e+00,
           -5.2924e-01,  1.0514e+00],
          ...,
     

In [ ]:
a+b

[[tensor(2), tensor(4), tensor(6)], [tensor(4), tensor(8), tensor(12)]]

In [ ]:
from typing import Any


class ML(ml.Base):
    def __init__(self):
        super().__init__()
        self.layer1 = super().Linear(3, 4, bias = True)
        self.layer2 = super().Linear(4, 4, bias = True)
        self.layer3 = super().Linear(4, 1, bias = True)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [ ]:
nn = ML()

[Neuron[weights = [tensor(0.5057231496067542), tensor(0.1216620088947794), tensor(0.37413267254290294)]], Neuron[weights = [tensor(0.32274828514938425), tensor(0.8242014036232774), tensor(0.8310820104059486)]], Neuron[weights = [tensor(0.18169951137056684), tensor(0.39658079312356365), tensor(0.8737790380516185)]], Neuron[weights = [tensor(0.3780109672714609), tensor(0.547539809007406), tensor(0.0764630992246228)]]]
[Neuron[weights = [tensor(0.18073417133545522), tensor(0.46411297286209063), tensor(0.15358891131025876), tensor(0.34202137547631695)]], Neuron[weights = [tensor(0.2042789071156733), tensor(0.39125433082371397), tensor(0.8837624882834365), tensor(0.6601226864038646)]], Neuron[weights = [tensor(0.8456079386695624), tensor(0.4704264165711104), tensor(0.03540277139736492), tensor(0.3120261318332975)]], Neuron[weights = [tensor(0.8612030616056859), tensor(0.127827189279494), tensor(0.8479855200291866), tensor(0.4916699578716818)]]]
[Neuron[weights = [tensor(0.9463882621589014),

In [ ]:
nn.parameters()

[tensor(0.7778737336548615),
 tensor(0.43618495382987177),
 tensor(0.7291986161810051),
 tensor(0.8325688981468586),
 tensor(0.33798180970577896),
 tensor(0.252110256648856),
 tensor(0.3224846061237012),
 tensor(0.4680669064084577),
 tensor(0.9269094571386111),
 tensor(0.8906419327276586),
 tensor(0.9272655304380233),
 tensor(0.26662373625743485),
 tensor(0.039052722265477624),
 tensor(0.3981814097289669),
 tensor(0.018113477041598935),
 tensor(0.807149041856858),
 tensor(0.0357563394278787),
 tensor(0.5543276507962437),
 tensor(0.4568460548609654),
 tensor(0.9519290016563606),
 tensor(0.2740325110557976),
 tensor(0.8274735949200924),
 tensor(0.8595688043969141),
 tensor(0.23045493954957064),
 tensor(0.5643246125057418),
 tensor(0.22847872165932392),
 tensor(0.7249734367121684),
 tensor(0.9792485302176943),
 tensor(0.08200461240212731),
 tensor(0.1651588529753325),
 tensor(0.7893948579118861),
 tensor(0.1655428227395458),
 tensor(0.8125828004067128),
 tensor(0.25818993500500054),
 tens

In [ ]:
len(nn.parameters())

41

In [ ]:
nn.parameters()

[tensor(5.484444802905695),
 tensor(-1.436491858111654),
 tensor(1.2229393404550777),
 tensor(-1.7712923739504067),
 tensor(2.240126823704002),
 tensor(-0.11696018109053158),
 tensor(1.3361106007868675),
 tensor(-0.6273492296917964),
 tensor(-2.4198228978142478),
 tensor(0.8545214101627816),
 tensor(0.4566643976176727),
 tensor(1.7183635446343215),
 tensor(-0.5445458561845967),
 tensor(0.10980829575694756),
 tensor(0.219902615240272),
 tensor(0.7753513772082665),
 tensor(0.6928793687281641),
 tensor(0.6152929991181239),
 tensor(-0.2898992389391686),
 tensor(0.04974667771848042),
 tensor(-0.40877438713046393),
 tensor(-0.13209477979615294),
 tensor(0.13512308491175565),
 tensor(1.0680815041596448),
 tensor(0.6660621203456785),
 tensor(1.2185746167789964),
 tensor(1.2220387459089073),
 tensor(0.5166150557711846),
 tensor(-0.30421693756265045),
 tensor(0.06576362089523842),
 tensor(-0.37473384385205727),
 tensor(1.0748797753404324),
 tensor(0.11347546938028358),
 tensor(0.5851815970774711

In [ ]:
(
    (1 * 0.5852710958610633 + 2 * 0.6781303523312047 + 3 * 0.40034283819762506)
    * 0.6995435519855825
    + (1 * 0.7146272948639855 + 2 * 0.5480047984103927 + 3 * 0.2490988051906703)
    * 0.44415290096365734
)

3.334471304055083

In [ ]:
a = nn([ml.Tensor(i) for i in [1, 2, 3]])
print(a)

[tensor(8.911500724243059)]


In [ ]:
nn([2.0, 3.0, -1.0])

tensor(9.428227052350968)

In [ ]:
xs = [[2.0, 5.0, -1.0], [2.0, 3.0, -4.0], [0.5, 1.0, 1.0], [1.0, 1.0, -1.0]]
ys = [1.0, -1.0, -1.0, 1.0]

ypred = [nn(x) for x in xs]
ypred

NameError: name 'nn' is not defined

In [ ]:
import numpy as np
a = np.array(1)

In [ ]:
ypred = [nn(x) for x in xs]
ypred
loss = sum((yg - yp) ** 2 for yg, yp in zip(ys, ypred))
loss

tensor(1.2975807250822883e-09)

In [ ]:
nn.zero_grad()

In [ ]:
loss.backward()

In [ ]:
for parameter in nn.parameters():
    parameter.data += -0.001 * parameter.grad

In [ ]:
for _ in range(1000):
    # forward propagation
    ypred = [nn(x) for x in xs]
    loss = sum((yg - yp) ** 2 for yg, yp in zip(ys, ypred))
    # backward propagation
    nn.zero_grad()
    loss.backward()
    # gradient descent
    for parameter in nn.parameters():
        parameter.data += -0.001 * parameter.grad

In [ ]:
nn.layer3.neurons[0].weights[0]

tensor(0.7610612369354476)

In [ ]:
nn.layer3.neurons[0].weights[0].grad

185.3270252234538

In [ ]:
import torch

x = torch.nn.Linear(3,2, bias=False)

In [ ]:
len(list(x.parameters()))

1

In [ ]:
list(x.parameters())

[Parameter containing:
 tensor([[-0.1338,  0.4814,  0.2520],
         [ 0.0121,  0.3811, -0.1008]], requires_grad=True)]